In [38]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import cv2

In [39]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
print(face_cascade)

<CascadeClassifier 0x7f3bae176db0>


In [40]:
 def face_extract(img):
    gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        cropped=img[y:y+h, x:x+w]
    
    return cropped


In [7]:
cap=cv2.VideoCapture(0)
count=0
while(True):
    ret, frame=cap.read()
    if face_extract(frame) is not None:
        count+=1
        
        img= cv2.resize(face_extract(frame), (200,200))
        #img= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        file_name_path='./Faces/Kashish/'+str(count)+'.jpg'
        cv2.imwrite(file_name_path, img)
        
        cv2.putText(img, str(count), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,0),2)
        cv2.imshow('image',img)
        if cv2.waitKey(13)& 0xFF == ord('q'):
            break
        
    if count==1000:
        break
        
cap.release()
cv2.destroyAllWindows()

In [41]:
train=[]
label=[]
names=[ 'Kashish', 'Rakshit', 'Aditya']
for name in names:
    for i in range(1,100):       
        photo=cv2.imread('./Faces/'+name+'/'+ str(i)+'.jpg')
        train.append(np.asarray(photo))
        label.append(name)
 

In [42]:
data=pd.DataFrame({'input':train, 'label':label})
data['label']=label
df = data.sample(frac=1).reset_index(drop=True)

In [43]:
y=np.array(df['label'])
y.shape

(297,)

In [44]:
z=[]
for i in range(df.shape[0]):
    z.append(df['input'][i])
z=np.array(z)
z.shape
#z.reshape(267,200,200)
#print(z)

(297, 200, 200, 3)

In [45]:
print(z.shape)

(297, 200, 200, 3)


In [46]:
X_train= z[:230]
X_train=X_train/255.0
X_test=z[230:]
X_test=X_test/255.0
y_train= y[:230]
y_test=y[230:]

In [47]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(230, 200, 200, 3)
(67, 200, 200, 3)
(230,)
(67,)


# Model Building

In [48]:
import keras
import time

In [49]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.callbacks import TensorBoard

In [50]:
names=['Kashish', 'Aditya', 'Rakshit']

In [51]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(names)
print(integer_encoded)

[1 0 2]


In [80]:
y_tr=label_encoder.transform(y_train)
y_te=label_encoder.transform(y_test)
#print(y_tr)
#print(y_te)
print(y_test)

['Rakshit' 'Kashish' 'Rakshit' 'Kashish' 'Kashish' 'Rakshit' 'Rakshit'
 'Rakshit' 'Kashish' 'Aditya' 'Kashish' 'Rakshit' 'Kashish' 'Rakshit'
 'Rakshit' 'Rakshit' 'Rakshit' 'Kashish' 'Rakshit' 'Kashish' 'Aditya'
 'Kashish' 'Kashish' 'Kashish' 'Kashish' 'Aditya' 'Kashish' 'Rakshit'
 'Rakshit' 'Aditya' 'Rakshit' 'Aditya' 'Aditya' 'Rakshit' 'Rakshit'
 'Rakshit' 'Aditya' 'Rakshit' 'Aditya' 'Kashish' 'Aditya' 'Rakshit'
 'Aditya' 'Rakshit' 'Aditya' 'Aditya' 'Aditya' 'Rakshit' 'Aditya' 'Aditya'
 'Kashish' 'Aditya' 'Aditya' 'Aditya' 'Rakshit' 'Aditya' 'Rakshit'
 'Rakshit' 'Kashish' 'Rakshit' 'Rakshit' 'Rakshit' 'Kashish' 'Kashish'
 'Aditya' 'Aditya' 'Rakshit']


In [59]:
from keras.utils import to_categorical
y_binary = to_categorical(y_tr)
y_binary_tst=to_categorical(y_te)
#print(y_binary)

In [54]:
model=Sequential()
name='cnn-{}'.format(int(time.time()))
model.add(Conv2D(64, (3,3), input_shape=(200,200,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='sigmoid'))
tensorboard = TensorBoard(log_dir="logs/{}".format(name))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(X_train, y_binary, batch_size=32, epochs=5, validation_split=0.3, callbacks=[tensorboard])

Train on 161 samples, validate on 69 samples
Epoch 1/5
161/161 [==============================] - 15s 92ms/step - loss: 1.3098 - acc: 0.3416 - val_loss: 0.9928 - val_acc: 0.2464
Epoch 2/5
161/161 [==============================] - 11s 68ms/step - loss: 0.7870 - acc: 0.5155 - val_loss: 0.9226 - val_acc: 0.6377
Epoch 3/5
161/161 [==============================] - 11s 66ms/step - loss: 0.5883 - acc: 0.7764 - val_loss: 0.6276 - val_acc: 0.6812
Epoch 4/5
161/161 [==============================] - 11s 66ms/step - loss: 0.2287 - acc: 0.8882 - val_loss: 0.0415 - val_acc: 0.9710
Epoch 5/5
161/161 [==============================] - 11s 66ms/step - loss: 0.0165 - acc: 1.0000 - val_loss: 0.0539 - val_acc: 0.9710


In [71]:
prediction=model.predict(X_test)
prediction.shape

(67, 3)

In [79]:
ans=[]
result=[]
for i in range(prediction.shape[0]):
    ans.append(np.argmax(prediction[i]))
print(ans)
result=label_encoder.inverse_transform(ans)
print(result)

[2, 1, 2, 1, 1, 2, 2, 2, 1, 0, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 0, 1, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 0, 1, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 2, 0, 2, 2, 1, 2, 2, 2, 1, 2, 0, 0, 2]
['Rakshit' 'Kashish' 'Rakshit' 'Kashish' 'Kashish' 'Rakshit' 'Rakshit'
 'Rakshit' 'Kashish' 'Aditya' 'Kashish' 'Rakshit' 'Kashish' 'Rakshit'
 'Rakshit' 'Rakshit' 'Rakshit' 'Kashish' 'Rakshit' 'Kashish' 'Aditya'
 'Kashish' 'Kashish' 'Kashish' 'Kashish' 'Aditya' 'Kashish' 'Rakshit'
 'Rakshit' 'Aditya' 'Rakshit' 'Aditya' 'Aditya' 'Rakshit' 'Rakshit'
 'Rakshit' 'Aditya' 'Rakshit' 'Aditya' 'Kashish' 'Aditya' 'Rakshit'
 'Aditya' 'Rakshit' 'Aditya' 'Aditya' 'Aditya' 'Rakshit' 'Aditya' 'Aditya'
 'Kashish' 'Aditya' 'Aditya' 'Aditya' 'Rakshit' 'Aditya' 'Rakshit'
 'Rakshit' 'Kashish' 'Rakshit' 'Rakshit' 'Rakshit' 'Kashish' 'Rakshit'
 'Aditya' 'Aditya' 'Rakshit']


In [61]:
val_loss, val_acc=model.evaluate(X_test, y_binary_tst)

67/67 [==============================] - 1s 14ms/step


In [62]:
print(val_loss, val_acc)

0.033622854319748596 0.9850746268656716


In [21]:
dense_layers=[0,1,2]
layer_sizes= [32, 64, 128]
conv_layers=[1,2,3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            name= "{}- conv-{}-nodes- {}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print("********************* "+name+" ************************")  
            
            model=Sequential()
            model.add(Conv2D(layer_size, (3,3), input_shape=X_train.shape[1:], activation='relu'))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for i in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3,3), activation='relu'))
                model.add(MaxPooling2D(pool_size=(2,2)))
                
            model.add(Flatten())
            for i in range(dense_layer):
                model.add(Dense(layer_size, activation='relu'))
            
            model.add(Dense(3, activation='softmax')) 
            tensorboard = TensorBoard(log_dir="logs/{}".format(name))
            model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
            model.fit(X_train, y_binary, batch_size=32, epochs=10, validation_split=0.3, callbacks=[tensorboard])

SyntaxError: invalid syntax (<ipython-input-21-8064839cafde>, line 27)

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 198, 198, 64)      1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 99, 99, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 97, 97, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 147456)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                9437248   
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total para